тут изначально 4 признака из текстовых полей датасета, сливаются в один и заполнялись пропуски, там где они были.

In [ ]:
data1['additional']= data1['additional'].fillna('или')
#датасет и так невелик, поэтому нашла выход чем нейтральным заполнить пропуски в "additional"

In [ ]:
data1['language'] = data1['mandatory']+data1['work_conditions']+data1['job_title']+data1['additional']


дальше при обучении была токенизация и ВЕRT. На всякий случай добавлю их, т.к. не очень уверена что именно может быть подключено в проде.




In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
tokenized = data1['language'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
batch_size = 100

In [ ]:
#Этот код ждать выполнения не обязательно, он считался в Colab около 4 часов, я выгрузила его результаты дальше как переменные

batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        #embeddings.append(batch_embeddings[0][:,:].numpy())
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
        del batch
        del attention_mask_batch
        del batch_embeddings



  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
features = np.concatenate(embeddings)
